In [11]:
using CSV, DataFrames, Plots, Random, DecisionTree, Statistics, LinearAlgebra, JuMP, Gurobi, NearestNeighbors

In [129]:
const GRB_ENV = Gurobi.Env(output_flag=0);

In [69]:
# load data, we have to keep the data for all the players
data = CSV.read("../data/fifa/players_22_preprocessed.csv", DataFrame)
transfers = CSV.read("../data/Transfers/all_transfers.csv", DataFrame);

In [41]:
# iterate over all clubs, and save players that are part of that clubs
club_players = Dict()
for club in Set(data[:, "club_name"])
    club_players[club] = data[data[:, "club_name"] .== club, :]
end

In [75]:
# keep only 2022 transfers
transfers = transfers[transfers[:, "year"] .== 2022, :];

In [77]:
transfers[1:5,:]

Row,club_name,player_name,age,position,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,year,season
,String,String,Float64?,String31,String31,String31?,String3,String7,Float64?,String31,Int64,String15
1,Bayern Munich,Matthijs de Ligt,22.0,Centre-Back,Juventus,€67.00m,in,summer,67.0,1 Bundesliga,2022,2022/2023
2,Bayern Munich,Sadio Mané,30.0,Left Winger,Liverpool,€32.00m,in,summer,32.0,1 Bundesliga,2022,2022/2023
3,Bayern Munich,Mathys Tel,17.0,Centre-Forward,Stade Rennais,€20.00m,in,summer,20.0,1 Bundesliga,2022,2022/2023
4,Bayern Munich,Ryan Gravenberch,20.0,Central Midfield,Ajax,€18.50m,in,summer,18.5,1 Bundesliga,2022,2022/2023
5,Bayern Munich,Noussair Mazraoui,24.0,Right-Back,Ajax,free transfer,in,summer,0.0,1 Bundesliga,2022,2022/2023


When we do prescription, we are pretending to be one specific club. We can do the same for multiple clubs.

In [124]:
club_choosen = "FC Barcelona"
club_choosen_transferName = "FC Barcelona"
;

In [125]:
data_club = club_players[club_choosen]
data_rest = data[data[:, "club_name"] .!= club_choosen, :]
;

In [126]:
# useful to compare our model with what truly happens
transfers_club = transfers[transfers[:, "club_name"] .== club_choosen_transferName, ["transfer_movement", "fee_cleaned"]]
transfers_club = transfers_club[completecases(transfers_club), :]

transfers_in = transfers_club[transfers_club[:, "transfer_movement"] .== "in", "fee_cleaned"]
transfers_out = transfers_club[transfers_club[:, "transfer_movement"] .== "out", "fee_cleaned"]

net_spend = sum(transfers_in) - sum(transfers_out)

118.0

In [178]:
# get parameters we need in our prescriptive model

n = size(data_rest, 1) # players that can be recruited
N_c = k = size(data_club, 1) # number of players in the club, equal to how many we can consider to sell
N_min = minimum([size(club_players[club])[1] for club in keys(club_players)]) # minimum number of players in the club
N_max = maximum([size(club_players[club])[1] for club in keys(club_players)]) # maximum number of players in the club
B_max = 10 # maximum number of players that can be bought
S_max = 10 # maximum number of players that can be sold

# for players outside the club
p_o = data_rest[:, "potential"] # potential
r_o = data_rest[:, "overall"] # rating
v_o = data_rest[:, "value_eur"] # value (later we will use the value predicted in the prediction part)
m_o = 0.9 * r_o + 0.1 * p_o # overall rating

# for players inside the club
p_c = data_club[:, "potential"] # potential 
r_c = data_club[:, "overall"] # rating 
v_c = data_club[:, "value_eur"] # value (same as above)
m_c = 0.9 * r_c + 0.1 * p_c # overall rating

𝛅 = 1 # how much we want to improve the team
;

In [179]:
sum(m_c) / k

77.61851851851851

# Approach 1: Regress and compare

In [188]:
function regress_and_compare(n, N_c, k, N_min, N_max, B_max, S_max, p_o, r_o, v_o, m_o, p_c, r_c, v_c, m_c, 𝛅)

    # define model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attributes(model, "OutputFlag" => 0, "TimeLimit" => 60)

    # define variables
    @variable(model, b[i = 1:n], Bin) # 1 if we buy player i, 0 otherwise
    @variable(model, s[j = 1:k], Bin) # 1 if we sell player j, 0 otherwise

    # define objective
    @objective(model, Max, sum(s[j] * v_c[j] for j = 1:k) - sum(b[i] * v_o[i] for i = 1:n))

    # define constraints
    @constraint(model, sum((1 - s[j]) * m_c[j] for j = 1:k) + sum(b[i] * m_o[i] for i = 1:n) >= (sum(m_c[j] for j = 1:k)/k + 𝛅) * (k + sum(b[i] for i = 1:n) - sum(s[j] for j = 1:k)))
    @constraint(model, sum(b[i] for i = 1:n) <= B_max) # maximum number of players that can be bought
    @constraint(model, sum(s[j] for j = 1:k) <= S_max) # maximum number of players that can be sold
    @constraint(model, N_min <= N_c + sum(b[i] for i = 1:n) - sum(s[j] for j = 1:k) <= N_max) # number of players in the club after the transfer window

    # solve model
    optimize!(model)

    return objective_value(model), value.(b), value.(s)
end

regress_and_compare (generic function with 3 methods)

In [189]:
obj, b, s = regress_and_compare(n, N_c, k, N_min, N_max, B_max, S_max, p_o, r_o, v_o, m_o, p_c, r_c, v_c, m_c, 𝛅);

In [193]:
Int(obj)

359700000

In [191]:
# get the players that we buy and sell
buy = data_rest[b .== 1, :]

Row,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_level,club_joined,club_contract_valid_until,preferred_foot,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,position_A,position_D,position_G,position_M,attack_work_rate,defense_work_rate
,String31,Int64,Int64,Float64,Float64,Int64,Int64,Int64,String,Float64,Int64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,G. Chiellini,86,86,1.2e7,88000.0,36,187,85,Juventus,1.0,2021,2023.0,0,3,2,4,1.98e7,68.0,46.0,60.0,59.0,89.0,85.0,63,33,77,65,45,57,60,31,67,57,67,69,53,80,57,78,91,71,91,49,88,89,28,50,50,83,93,89,88,3,3,2,4,3,0,1,0,0,2,3
2,Thiago Silva,85,85,9.5e6,105000.0,36,183,79,Chelsea,1.0,2021,2022.0,1,3,2,4,1.76e7,53.0,54.0,72.0,72.0,86.0,78.0,60,40,81,80,61,67,62,61,80,79,52,54,67,82,66,71,88,68,82,65,76,88,59,71,60,86,87,86,84,9,12,5,9,10,0,1,0,0,2,3
3,Fernandinho,83,83,7.0e6,100000.0,36,179,67,Manchester City,1.0,2021,2022.0,1,4,3,3,1.3e7,59.0,72.0,75.0,78.0,84.0,75.0,68,66,71,83,70,76,68,71,81,81,67,52,69,86,77,83,84,66,75,76,86,85,70,66,61,81,87,85,80,12,11,5,13,7,0,0,0,1,2,3
4,Pepe,82,82,5.5e6,14000.0,38,188,80,FC Porto,1.0,2021,2023.0,1,3,2,4,1.1e7,80.0,51.0,60.0,61.0,82.0,87.0,46,46,81,73,23,58,44,47,75,60,79,81,62,81,48,63,89,82,86,56,94,78,40,48,57,80,86,82,81,8,15,5,9,10,0,1,0,0,2,3
5,José Fonte,81,81,4.6e6,30000.0,37,191,84,LOSC Lille,1.0,2021,2022.0,1,3,2,3,8.7e6,32.0,46.0,62.0,63.0,83.0,77.0,47,42,85,74,36,61,51,32,69,68,32,32,43,83,51,60,74,57,84,43,83,82,48,63,43,79,84,83,78,7,11,16,10,6,0,1,0,0,2,3


In [192]:
sell = data_club[s .== 1, :]

Row,short_name,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_name,league_level,club_joined,club_contract_valid_until,preferred_foot,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,position_A,position_D,position_G,position_M,attack_work_rate,defense_work_rate
,String31,Int64,Int64,Float64,Float64,Int64,Int64,Int64,String,Float64,Int64,Float64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,S. Agüero,87,87,5.1e7,260000.0,33,173,70,FC Barcelona,1.0,2021,2023.0,1,4,4,4,1.046e8,71.0,89.0,75.0,87.0,33.0,69.0,70,93,78,80,85,86,83,73,64,88,75,68,82,89,90,90,78,62,74,83,65,24,91,80,75,91,30,29,24,13,15,6,11,14,1,0,0,0,3,2
2,F. de Jong,87,92,1.195e8,210000.0,24,180,74,FC Barcelona,1.0,2021,2026.0,1,3,4,3,2.539e8,81.0,69.0,85.0,88.0,77.0,78.0,78,72,72,91,70,88,84,64,87,89,80,82,88,88,79,68,76,90,74,66,75,82,77,86,45,90,76,76,77,7,13,10,10,9,0,0,0,1,3,2
3,Jordi Alba,86,86,4.7e7,200000.0,32,170,68,FC Barcelona,1.0,2021,2024.0,0,3,3,3,9.64e7,86.0,69.0,81.0,83.0,77.0,71.0,85,73,70,85,60,81,80,63,79,85,87,86,90,80,85,64,80,87,61,66,75,81,83,78,59,80,74,78,80,13,15,13,6,13,0,1,0,0,3,2
4,M. Depay,85,86,6.3e7,220000.0,27,176,78,FC Barcelona,1.0,2021,2023.0,1,3,5,3,1.339e8,82.0,83.0,82.0,84.0,30.0,79.0,84,83,65,81,74,86,85,83,76,85,81,83,80,83,81,87,69,79,83,83,72,28,85,85,76,80,23,31,20,8,14,6,12,10,1,0,0,0,3,1
5,O. Dembélé,83,88,5.5e7,165000.0,24,178,67,FC Barcelona,1.0,2021,2022.0,0,5,5,3,1.169e8,93.0,77.0,77.0,86.0,36.0,56.0,81,75,43,78,76,89,82,64,76,84,94,93,89,78,80,83,58,68,51,76,54,36,77,76,75,80,42,30,33,6,6,14,10,13,1,0,0,0,3,2
6,Pedri,81,91,5.4e7,51000.0,18,174,61,FC Barcelona,1.0,2021,2022.0,1,4,4,2,1.215e8,78.0,65.0,79.0,84.0,66.0,63.0,68,68,50,85,45,84,73,51,80,83,81,76,88,81,89,62,65,86,53,65,59,68,77,86,53,84,65,73,63,12,7,11,8,8,0,0,0,1,3,3
7,Matheus Pereira,68,76,2.7e6,35000.0,23,181,68,FC Barcelona,1.0,2021,2025.0,0,3,4,1,6.1e6,69.0,60.0,65.0,74.0,35.0,58.0,60,62,52,67,66,75,73,64,63,74,68,69,78,70,69,58,59,69,55,57,50,33,62,67,62,73,25,40,38,9,13,7,9,10,0,0,0,1,2,2
8,K. Ruiz-Atil,66,81,2.0e6,10000.0,18,173,60,FC Barcelona,1.0,2021,2024.0,1,3,3,1,5.4e6,67.0,54.0,64.0,72.0,44.0,41.0,54,53,30,70,52,72,68,59,65,71,69,66,81,59,79,60,50,59,32,48,40,29,60,64,58,69,43,55,53,8,8,14,13,9,0,0,0,1,2,2
9,Álex Balde,66,82,2.0e6,1000.0,17,175,69,FC Barcelona,1.0,2021,2024.0,0,3,3,1,5.4e6,76.0,35.0,53.0,66.0,62.0,48.0,67,40,58,60,32,68,31,25,42,69,74,78,59,62,59,44,34,62,46,14,38,64,40,45,41,47,65,61,60,5,8,7,10,13,0,1,0,0,3,2


# see whatsapp messages for possible improvements